In [ ]:
!pip install -U "transformers>=4.34.0" "datasets>=2.15.0" "accelerate>=0.21.0" \
            "torch>=2.0.0" "pandas>=2.0.0" "numpy>=1.24.0" "scikit-learn>=1.3.0" \
            "joblib>=1.3.0" "pyarrow>=14.0.0" "fastparquet>=2023.8.0" "tqdm>=4.66.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer
import pandas as pd

def load_parquet_dataset(parquet_path, text_field="code", label_field="label"):
    """
    Load a parquet file as a streaming Hugging Face dataset.
    Returns an IterableDataset that can be sampled from.
    """
    dataset = load_dataset("parquet", data_files=parquet_path, split="train", streaming=True)
    return dataset.map(lambda x: {text_field: x[text_field], label_field: x[label_field]})


class CodeDatasetPreprocessor:
    """
    Tokenizes code or text fields in Hugging Face datasets for transformer models.
    """

    def __init__(self, model_name_or_path, max_length=512, text_field="code"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.max_length = max_length
        self.text_field = text_field

    def tokenize_batch(self, examples):
        return self.tokenizer(
            examples[self.text_field],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )

    def prepare(self, dataset):
        """
        Apply tokenization to a Hugging Face Dataset (non-streaming).
        """
        return dataset.map(
            self.tokenize_batch,
            batched=True,
            remove_columns=[self.text_field]
        )


In [ ]:
# src/metrics.py
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import math
from typing import Dict

def classification_metrics(preds, labels) -> Dict:
    """Compute accuracy, precision, recall, and f1 (weighted).
    preds can be logits (ndarray) or 1d predicted labels."""
    import numpy as _np
    preds_arr = _np.array(preds)
    if preds_arr.ndim > 1:
        pred_labels = preds_arr.argmax(axis=1)
    else:
        pred_labels = preds_arr
    acc = accuracy_score(labels, pred_labels)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, pred_labels, average='weighted', zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

def perplexity(loss):
    return math.exp(loss)

In [ ]:
# src/models.py
from transformers import AutoModelForSequenceClassification, AutoConfig

def get_sequence_classification_model(name_or_path, num_labels, from_pretrained=True):
    if from_pretrained:
        model = AutoModelForSequenceClassification.from_pretrained(name_or_path, num_labels=num_labels)
    else:
        config = AutoConfig.from_pretrained(name_or_path, num_labels=num_labels)
        model = AutoModelForSequenceClassification.from_config(config)
    return model

In [ ]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None


In [ ]:
%reset -f

In [ ]:
!kill -9 -1

In [ ]:
from datasets import disable_caching
disable_caching()

!rm -rf ~/.cache/huggingface

In [ ]:
import os
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"

In [ ]:
# ==============================
# ✅ SETUP
# ==============================
import os
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset, IterableDataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import math

# Disable unnecessary parallelism warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"


# ==============================
# ✅ DATA HANDLING
# ==============================
def load_parquet_dataset(parquet_path, text_field="code", label_field="label"):
    """
    Load a Parquet dataset in memory (not streaming).
    Works with Hugging Face datasets.
    """
    dataset = load_dataset("parquet", data_files=parquet_path)["train"]

    # Ensure correct field names exist
    sample = dataset[0]
    if text_field not in sample:
        raise ValueError(f"'{text_field}' not found in dataset columns: {list(sample.keys())}")
    if label_field not in sample:
        raise ValueError(f"'{label_field}' not found in dataset columns: {list(sample.keys())}")

    return dataset


def subsample_dataset(dataset, max_size=6000, seed=42):
    """Subsample a Hugging Face dataset or pandas DataFrame to max_size."""
    if isinstance(dataset, IterableDataset):
        print("⚠️ IterableDataset detected — converting to in-memory dataset to allow subsampling.")
        dataset = list(dataset)
        dataset = Dataset.from_list(dataset)

    if len(dataset) > max_size:
        try:
            dataset = dataset.shuffle(seed=seed).select(range(max_size))
        except AttributeError:
            dataset = dataset.sample(n=max_size, random_state=seed)
    return dataset


# ==============================
# ✅ TOKENIZATION / PREPROCESSING
# ==============================
class CodeDatasetPreprocessor:
    """
    Tokenizes code or text fields in Hugging Face datasets for transformer models.
    """
    def __init__(self, model_name_or_path, max_length=256, text_field="code"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.max_length = max_length
        self.text_field = text_field

    def tokenize_batch(self, examples):
        return self.tokenizer(
            examples[self.text_field],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )

    def prepare(self, dataset):
        return dataset.map(
            self.tokenize_batch,
            batched=True,
            remove_columns=[self.text_field]
        )


# ==============================
# ✅ MODEL & METRICS
# ==============================
def get_sequence_classification_model(name_or_path, num_labels, from_pretrained=True):
    if from_pretrained:
        model = AutoModelForSequenceClassification.from_pretrained(
            name_or_path, num_labels=num_labels
        )
    else:
        from transformers import AutoConfig
        config = AutoConfig.from_pretrained(name_or_path, num_labels=num_labels)
        model = AutoModelForSequenceClassification.from_config(config)
    return model


def classification_metrics(preds, labels):
    preds_arr = np.array(preds)
    if preds_arr.ndim > 1:
        pred_labels = preds_arr.argmax(axis=1)
    else:
        pred_labels = preds_arr
    acc = accuracy_score(labels, pred_labels)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, pred_labels, average="weighted", zero_division=0
    )
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return classification_metrics(np.array(logits), np.array(labels))


# ==============================
# ✅ MAIN TRAINING FUNCTION
# ==============================
def main():
    # Configuration
    model_name = "Salesforce/codet5-base"     # ✅ small enough for Colab
    train_path = "/content/drive/MyDrive/trainingset.parquet"
    val_path = "/content/drive/MyDrive/validationset.parquet"   # replace with real val file
    out_dir = "/content/drive/MyDrive/experiments"
    num_labels = 11
    epochs = 3
    batch_size = 8
    lr = 3e-5
    os.makedirs(out_dir, exist_ok=True)
    # --- Load datasets ---
    print("📂 Loading parquet datasets...")
    train_ds = load_parquet_dataset(train_path)
    val_ds = load_parquet_dataset(val_path)

    # --- Subsample (optional) ---
    print("📉 Subsampling datasets (if large)...")
    train_ds = subsample_dataset(train_ds, max_size=6000)
    val_ds = subsample_dataset(val_ds, max_size=2000)

    # --- Tokenization ---
    print("🔧 Tokenizing datasets...")
    preproc = CodeDatasetPreprocessor(model_name, max_length=256, text_field="code")
    train_tok = preproc.prepare(train_ds)
    val_tok = preproc.prepare(val_ds)

    # --- Model ---
    print("🤖 Initializing model...")
    model = get_sequence_classification_model(model_name, num_labels=num_labels)

    # --- Training args ---
    args = TrainingArguments(
        output_dir=out_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_steps=50,
        fp16=True,                # ✅ Enable half precision if Colab GPU supports it
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        report_to="none"          # avoid TensorBoard in Colab
    )

    # --- Trainer ---
    data_collator = DataCollatorWithPadding(tokenizer=preproc.tokenizer)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tok,
        eval_dataset=val_tok,
        tokenizer=preproc.tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # --- Train ---
    print("🚀 Starting training...")
    trainer.train()
    trainer.save_model(out_dir)
    preproc.tokenizer.save_pretrained(out_dir)
    print(f"✅ Training complete. Model saved to {out_dir}")
if __name__ == "__main__":
    main()


In [ ]:
# ==============================
# ✅ SETUP
# ==============================
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset, IterableDataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# ==============================
# ✅ DATA HANDLING
# ==============================
def load_parquet_dataset(parquet_path, text_field="code", label_field="label"):
    dataset = load_dataset("parquet", data_files=parquet_path)["train"]
    sample = dataset[0]
    if text_field not in sample:
        raise ValueError(f"'{text_field}' not found in dataset columns: {list(sample.keys())}")
    if label_field not in sample:
        raise ValueError(f"'{label_field}' not found in dataset columns: {list(sample.keys())}")
    return dataset


def subsample_dataset(dataset, max_size=6000, seed=42):
    if isinstance(dataset, IterableDataset):
        dataset = Dataset.from_list(list(dataset))
    if len(dataset) > max_size:
        dataset = dataset.shuffle(seed=seed).select(range(max_size))
    return dataset


# ==============================
# ✅ TOKENIZATION
# ==============================
class CodeDatasetPreprocessor:
    def __init__(self, model_name_or_path, max_length=256, text_field="code"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
        self.max_length = max_length
        self.text_field = text_field

    def tokenize_batch(self, examples):
        return self.tokenizer(
            examples[self.text_field],
            truncation=True,
            padding="max_length",
            max_length=self.max_length
        )

    def prepare(self, dataset):
        return dataset.map(self.tokenize_batch, batched=True, remove_columns=[self.text_field])


# ==============================
# ✅ METRICS
# ==============================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted", zero_division=0
    )
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}


# ==============================
# ✅ MAIN TRAINING FUNCTION
# ==============================
def main():
    model_name = "microsoft/codebert-base"   # ✅ lighter than CodeT5 for Colab
    train_path = "/content/drive/MyDrive/trainingset.parquet"
    val_path = "/content/drive/MyDrive/validationset.parquet"
    out_dir = "/content/drive/MyDrive/experiments"
    num_labels = 11
    epochs = 3
    batch_size = 8
    lr = 3e-5

    os.makedirs(out_dir, exist_ok=True)

    print("📂 Loading parquet datasets...")
    train_ds = load_parquet_dataset(train_path)
    val_ds = load_parquet_dataset(val_path)

    print("📉 Subsampling datasets (if large)...")
    train_ds = subsample_dataset(train_ds, max_size=6000)
    val_ds = subsample_dataset(val_ds, max_size=2000)

    print("🔧 Tokenizing datasets...")
    preproc = CodeDatasetPreprocessor(model_name, max_length=256, text_field="code")
    train_tok = preproc.prepare(train_ds)
    val_tok = preproc.prepare(val_ds)

    print("🤖 Initializing model...")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    args = TrainingArguments(
        output_dir=out_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_steps=50,
        fp16=True,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        report_to="none"
    )

    data_collator = DataCollatorWithPadding(tokenizer=preproc.tokenizer)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tok,
        eval_dataset=val_tok,
        tokenizer=preproc.tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )
    print("🚀 Starting training...")
    trainer.train()
    trainer.save_model(out_dir)
    preproc.tokenizer.save_pretrained(out_dir)
    print(f"✅ Training complete. Model saved to {out_dir}")

if __name__ == "__main__":
    main()
